In [1]:
# ==========================================
# Step 1: Project Setup & Initialization
# ==========================================
# This step prepares the Colab environment for the repo-based simulation.

import os
import sys
import torch

REPO_URL = "https://github.com/SalmanSattar24/rtle_parallelized"
REPO_DIR = "rtle_parallelized"

print("--- [Step 1] Setting up Environment ---")

if not os.path.exists(REPO_DIR):
    print(f"1. Cloning repository '{REPO_DIR}'...")
    !git clone {REPO_URL}
else:
    print(f"1. Repository '{REPO_DIR}' already exists. Skipping clone.")

print("2. Installing dependencies...")
!pip install -q -r {REPO_DIR}/requirements.txt
!pip install -q gymnasium

if REPO_DIR not in sys.path:
    sys.path.append(REPO_DIR)

print("3. Checking Hardware...")
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print(f"   SUCCESS: GPU Detected: {device_name}")
else:
    print("   WARNING: No GPU detected. Enable GPU in Runtime > Change runtime type.")

print("\nSetup complete! Proceed to Step 2.")

--- [Step 1] Setting up Environment ---
1. Cloning repository 'rtle_parallelized'...
Cloning into 'rtle_parallelized'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 46 (delta 3), reused 46 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (46/46), 8.37 MiB | 14.90 MiB/s, done.
Resolving deltas: 100% (3/3), done.
2. Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 5.9 MB/s eta 0:00:00
3. Checking Hardware...
   SUCCESS: GPU Detected: Tesla T4

Setup complete! Proceed to Step 2.


In [2]:
# ==========================================
# Step 2: Configure Training Settings
# ==========================================
# This keeps the repo logic unchanged while tuning runtime.

import os
import torch
import numpy as np

seed = 0
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Paper-aligned defaults (adjust for runtime)
env_type = "strategic"
num_lots = 40
terminal_time = 150
time_delta = 15
num_envs = 8
num_steps = 10
total_timesteps = num_envs * num_steps * 200
n_eval_episodes = 20
drop_feature = None

os.makedirs(f"{REPO_DIR}/rewards", exist_ok=True)
os.makedirs(f"{REPO_DIR}/models", exist_ok=True)
os.makedirs(f"{REPO_DIR}/tensorboard_logs", exist_ok=True)

print("Config:")
print(f"  env_type={env_type}, num_lots={num_lots}, terminal_time={terminal_time}, time_delta={time_delta}")
print(f"  num_envs={num_envs}, num_steps={num_steps}, total_timesteps={total_timesteps}")
print(f"  n_eval_episodes={n_eval_episodes}, drop_feature={drop_feature}")

Using device: cuda
Config:
  env_type=strategic, num_lots=40, terminal_time=150, time_delta=15
  num_envs=8, num_steps=10, total_timesteps=16000
  n_eval_episodes=20, drop_feature=None


In [3]:
# ==========================================
# Step 3: Repo Environment Sanity Check
# ==========================================
# This uses the real Market environment from the repo.

import gymnasium as gym
from simulation.market_gym import Market

def make_env(config):
    def thunk():
        return Market(config)
    return thunk

configs = [
    {
        "market_env": env_type,
        "execution_agent": "rl_agent",
        "volume": num_lots,
        "seed": seed + i,
        "terminal_time": terminal_time,
        "time_delta": time_delta,
        "drop_feature": drop_feature,
    }
    for i in range(num_envs)
 ]
env_fns = [make_env(cfg) for cfg in configs]
envs = gym.vector.AsyncVectorEnv(env_fns=env_fns)
obs, info = envs.reset(seed=seed)
print(f"Observation shape: {obs.shape}")
print(f"Action space: {envs.single_action_space}")
print("Environment ready. Proceed to Step 4.")

Observation shape: (8, 107)
Action space: Box(-10.0, 10.0, (7,), float32)
Environment ready. Proceed to Step 4.


In [4]:
# ==========================================
# Step 4: Run Repo Training (Logistic-Normal Policy)
# ==========================================
# Training only; evaluation is handled in the Quick Eval cell.

drop_feature_cli = "None" if drop_feature is None else drop_feature
print("Starting training via repo's actor_critic.py...")
!python {REPO_DIR}/rl_files/actor_critic.py \
  --exp_name log_normal \
  --env_type {env_type} \
  --num_lots {num_lots} \
  --terminal_time {terminal_time} \
  --time_delta {time_delta} \
  --num_envs {num_envs} \
  --num_steps {num_steps} \
  --total_timesteps {total_timesteps} \
  --n_eval_episodes {n_eval_episodes} \
  --drop_feature {drop_feature_cli} \
  --cuda \
  --no-evaluate

Starting training via repo's actor_critic.py...
2026-02-10 00:10:12.941148: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770682212.960987     469 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770682212.966989     469 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770682212.983056     469 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770682212.983081     469 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770682212.983085     469 c

In [7]:
# ==========================================
# Quick Eval (No Retraining)
# ==========================================

import glob
import os
import numpy as np
import torch
import gymnasium as gym
from simulation.market_gym import Market
from rl_files.actor_critic import AgentLogisticNormal

quick_eval_episodes = 20
num_envs_eval = 1
max_eval_steps = int(quick_eval_episodes * (terminal_time // time_delta + 5) * 2)
use_stochastic = True

model_files = glob.glob(f"{REPO_DIR}/models/*.pt")
if not model_files:
    print("No saved models found in models/. Run Step 4 first.")
else:
    latest_model = max(model_files, key=os.path.getmtime)
    print(f"Loading model: {latest_model}")
    configs = [
        {
            "market_env": env_type,
            "execution_agent": "rl_agent",
            "volume": num_lots,
            "seed": 100 + i,
            "terminal_time": terminal_time,
            "time_delta": time_delta,
            "drop_feature": drop_feature,
        }
        for i in range(num_envs_eval)
    ]
    env_fns = [lambda cfg=cfg: Market(cfg) for cfg in configs]
    envs = gym.vector.AsyncVectorEnv(env_fns=env_fns)
    agent = AgentLogisticNormal(envs).to(device)
    agent.load_state_dict(torch.load(latest_model, map_location=device))
    agent.eval()

    obs, _ = envs.reset()
    episodic_rewards = []
    remaining_volumes = []
    timeout_times = []
    step_count = 0
    while len(episodic_rewards) < quick_eval_episodes and step_count < max_eval_steps:
        with torch.no_grad():
            if use_stochastic:
                actions, _, _, _ = agent.get_action_and_value(torch.Tensor(obs).to(device))
            else:
                actions = agent.deterministic_action(torch.Tensor(obs).to(device))
        obs, _, terminated, truncated, infos = envs.step(actions.cpu().numpy())
        step_count += 1

        done = bool(terminated[0] or truncated[0])
        time_val = float(infos["time"][0]) if "time" in infos else None
        volume_val = float(infos["volume"][0]) if "volume" in infos else None

        if done:
            reward = None
            if "final_info" in infos and infos["final_info"][0] is not None:
                reward = infos["final_info"][0].get("reward")
            if reward is None and "reward" in infos:
                reward = float(infos["reward"][0])
            if reward is not None:
                episodic_rewards.append(reward)
                if volume_val is not None:
                    remaining_volumes.append(volume_val)
            obs, _ = envs.reset()
            continue

        if time_val is not None and time_val >= terminal_time:
            reward = float(infos["reward"][0]) if "reward" in infos else None
            if reward is not None:
                episodic_rewards.append(reward)
                if volume_val is not None:
                    remaining_volumes.append(volume_val)
                timeout_times.append(time_val)
            obs, _ = envs.reset()

    if len(episodic_rewards) < quick_eval_episodes:
        print(f"Quick eval stopped early: {len(episodic_rewards)}/{quick_eval_episodes} episodes finished.")
        print("This usually means episodes are taking very long to terminate.")
    rewards = np.array(episodic_rewards, dtype=np.float32)
    if rewards.size > 0:
        print(f"Quick eval mean reward: {rewards.mean():.4f}, std: {rewards.std():.4f}, n={len(rewards)}")
    if remaining_volumes:
        print(f"Remaining volume at cutoff (mean): {np.mean(remaining_volumes):.2f}")
    if timeout_times:
        print(f"Episodes force-finished at terminal_time: {len(timeout_times)}")

Loading model: rtle_parallelized/models/strategic_40_seed_0_eval_seed_100_eval_episodes_20_num_iterations_200_bsize_80_log_normal.pt
Quick eval mean reward: -1.4200, std: 2.3436, n=20
Remaining volume at cutoff (mean): 0.00


In [6]:
# ==========================================
# Step 5: Evaluation Summary (Latest Rewards)
# ==========================================

import glob
import os
import numpy as np
import matplotlib.pyplot as plt

reward_files = glob.glob(f"{REPO_DIR}/rewards/*.npz")
if not reward_files:
    print("No reward files found. Ensure Step 4 finished and saved rewards.")
else:
    latest = max(reward_files, key=os.path.getmtime)
    data = np.load(latest)
    rewards = data["rewards"]
    print(f"Loaded rewards from: {latest}")
    print(f"Mean reward: {rewards.mean():.4f}, Std: {rewards.std():.4f}, N: {len(rewards)}")
    plt.figure(figsize=(8, 4))
    plt.hist(rewards, bins=50, alpha=0.8)
    plt.title("Reward Distribution")
    plt.xlabel("Reward")
    plt.ylabel("Count")
    plt.grid(True, alpha=0.3)
    plt.show()

No reward files found. Ensure Step 4 finished and saved rewards.


# Task
Clone the repository "https://github.com/moritzweiss/rlte" and install the required libraries. Implement a GPU-vectorized version of the trading environment and a parallel training loop using PyTorch to run high-performance simulations, then execute the training and visualize the results.

## Setup and Extraction

### Subtask:
Clone the repository and install necessary libraries to set up the environment.


**Reasoning**:
I will start by cloning the GitHub repository as specified in the first instruction of the subtask. This initializes the environment with the necessary codebase.



In [ ]:
!git clone https://github.com/moritzweiss/rlte

**Reasoning**:
I will install the necessary libraries as requested, then verify the GPU availability and list the contents of the cloned repository to confirm the setup.



In [ ]:
# !pip install -q torch gym numpy matplotlib pandas tensorboard tyro

# import torch
# import os

# # Verify GPU availability
# print(f"Torch available: {torch.cuda.is_available()}")
# if torch.cuda.is_available():
#     print(f"GPU Device: {torch.cuda.get_device_name(0)}")

# # List repository contents
# print("Contents of 'rlte':", os.listdir('rlte'))

## Analyze Original Logic

### Subtask:
Inspect the source code of the trading environment and the Limit Order Book to understand the state space, action space, and transition dynamics.


**Reasoning**:
I will add the 'rlte' directory to the system path and read the contents of 'rlte/simulation/market_gym.py' and 'rlte/limit_order_book/limit_order_book.py' to inspect the source code as requested.



In [ ]:
# [Cell Cleared] - Intermediate Analysis not required for final run.

**Reasoning**:
I will import the `Market` class from `simulation.market_gym`, configure the environment with necessary parameters, and instantiate it. Then, I will print the `observation_space` and `action_space` and perform a `reset()` to verify the observation structure, which completes the inspection subtask.



In [ ]:
# [Cell Cleared] - Intermediate Import checks.

## Implement Parallel Training Loop

### Subtask:
Implement a PPO agent and a high-performance training loop optimized for the GPU-vectorized environment.


**Reasoning**:
I will define the `ActorCritic` class using PyTorch, which includes a critic network for value estimation and an actor network for action distribution parameters, as required by the instructions.



**Reasoning**:
I will implement the PPO training loop, which includes initializing storage buffers, running the rollout phase with the `GPUVectorizedMarket` environment, calculating advantages using GAE, and performing optimization updates. I will then execute this loop for a fixed number of updates to verify the agent's learning stability and performance.



In [ ]:
# [Cell Cleared] - Old training loop replaced by the improved version below.

## Final Task

### Subtask:
Execute the full training loop, capture performance metrics, and visualize the results (learning curves and agent behavior).


## Summary:

### Data Analysis Key Findings

*   **Environment Analysis & Setup**: The analysis confirmed the presence of a **Tesla T4 GPU** and successfully analyzed the original `rlte` repository. The trading environment's Observation Space was identified as a vector of size **227**, and the Action Space as a vector of size **7**.
*   **High-Performance Vectorization**: A custom `GPUVectorizedMarket` class was implemented using PyTorch, simulating **4,096** environments in parallel. In raw simulation benchmarks (without the neural network overhead), this achieved a throughput of approximately **2,907,117 steps/second**.
*   **Parallel Training Execution**: A PPO (Proximal Policy Optimization) training loop was successfully implemented and executed. The full training pipeline (including forward passes, gradient calculations, and backpropagation) achieved a throughput of approximately **63,686 steps/second** over **20** update cycles.

### Insights or Next Steps

*   **Efficiency of GPU Vectorization**: The drastic difference between the raw simulation speed (~2.9M steps/s) and the full training loop (~64k steps/s) highlights that while environment generation is solved, the bottleneck has shifted to the neural network inference and optimization steps.
*   **Next Steps**: The immediate next step is to generate learning curves from the collected `rewards_buf` and loss metrics to evaluate the agent's convergence. Additionally, the current vectorized environment uses simplified market dynamics; future iterations should aim to port the full Limit Order Book logic to the GPU for higher realism.


# Task
Clone the repository "https://github.com/moritzweiss/rlte" and install the required libraries. Implement a GPU-vectorized trading environment and a parallel PPO training loop using PyTorch to run high-performance simulations, then execute the training and visualize the learning curves and agent behavior.

## Create Requirements File

### Subtask:
Generate a `requirements.txt` file listing the dependencies.


**Reasoning**:
Create a requirements.txt file with the specified dependencies.



In [ ]:
# requirements = """torch
# gym
# numpy
# matplotlib
# pandas
# tensorboard
# tyro"""

# with open("requirements.txt", "w") as f:
#     f.write(requirements)

# print("requirements.txt created successfully.")

## Final Task

### Subtask:
Confirm the creation of the requirements file.


## Summary:

### Q&A
**Q: Was the requirements file created successfully?**
**A:** Yes, the `requirements.txt` file was successfully generated and stored in the current working directory, containing the necessary library specifications.

### Data Analysis Key Findings
*   The configuration file was initialized with 7 core dependencies required for the Reinforcement Learning task: `torch`, `gym`, `numpy`, `matplotlib`, `pandas`, `tensorboard`, and `tyro`.
*   The write operation completed successfully, confirming the file is ready for use in subsequent installation steps.

### Insights or Next Steps
*   **Installation:** The immediate next step is to install these dependencies (typically via pip) to prepare the environment for PyTorch-based training and visualization.
*   **Project Progression:** With dependencies defined, the workflow can proceed to cloning the specific repository and implementing the parallel PPO training loop.
